In [1]:
'''
TSP using genetic algorithm
'''

# Hardik Gandhi
# BT17CSE030

'\nTSP using genetic algorithm\n'

In [2]:
# library imports

import random 
from collections import defaultdict
import numpy as np
import sys
import os

In [3]:
def adding_new_edge_to_graph(graph, node1, node2, cost):
    
    '''
    Adding a new edge to the graph
    Input Parameters:
        graph : The actual graph (2D Dictionary) :: {'node1' : [(node2, cost2), (node3, cost3), ... ], 'node2' : [ ... ], ...}
        node1 : Source node of the edge (starting point)
        node2 : Destination node of the edge (end point)
        cost  : Cost to travel from node1 to node2
    '''
    
    graph[node1][node2] = cost
    graph[node2][node1] = cost

In [4]:
def initialize_population(arr, state, size):
    
    '''
    Initialize the population of size 'size', where each chromosome is basically one permutation of all the nodes
    here a string of cities
    Parameters:
        arr   : List of graph nodes
        state : To store the chromosomes
        size  : Size of the population
    '''
    
    for i in range(size):
        permutation = np.random.permutation(arr)
        state.append(list(permutation))

In [5]:
def fitness_value(graph, chromosome):
    
    '''
    Calculates fitness value of one chromosome
    fitness value is sum of distances between nodes
    Parameters:
        graph      : The actual graph of cities
        chromosome : One particular chromosome from the population
    Returns:
        The fitness value of i th chromosome
    '''
    
    ret, length = 0, len(chromosome)
    for i in range(length-1):
        ret = ret + graph[chromosome[i]][chromosome[i+1]]

    ret = ret + graph[chromosome[-1]][chromosome[0]] #returning distance from last node to first
    ret=1/ret
#     print(ret)
    return ret

In [6]:
def roulette_wheel_selection(fitness_array, max_parent):
    
    '''
    Select two parents from the population based upon fitness value, between which crossover is to be performed to create next generation child
    Parameters:
    fitness_array : An array consisting of fitness value of each chromosome in the population
    Returns:
    Two chosen parents
    this is based upon: fitter individual has a greater pie on the wheel and therefore a greater chance of selecting
    '''
    
    total_fitness = sum(fitness_array)
    roulette_wheel = [0.0]
# making the wheel
    for fit in fitness_array:
        new_val = roulette_wheel[-1] + (fit / total_fitness)
        roulette_wheel.append(new_val)

    no_parent, length, parents = 0, len(roulette_wheel), []
    
    while no_parent != max_parent:
        prob = np.random.random()
# selecting the sector
        for i in range(1, length):
            if prob >= roulette_wheel[i-1] and prob <= roulette_wheel[i]:
                p = i - 1
                if p not in parents:
                    parents.append(i-1)
                    no_parent = no_parent + 1

                break

    return parents

In [7]:
def crossover(state, parents, pop_size):
    
    '''
    Performs a crossover operation between parents, and replace the parents with childs in population
    Parameters:
        state    : Population
        parents  : here two parents selected by roulette-wheel-selection process
        pop_size : Size of the population
    '''
#     like a age based selection when a parent is selected for reproduction it is than replaces by its offsprings

    no_nodes, no_parents = len(state[0]), len(parents)
    for i in range(0, no_parents-1, 2):
        cross_point = np.random.randint(1,no_nodes//2)

        offspring1 = state[parents[i]].copy()
        for j in range(cross_point):
            c_index = state[parents[i]].index(state[parents[i+1]][j])
            offspring1[j], offspring1[c_index] = offspring1[c_index], offspring1[j]

        offspring2 = state[parents[i+1]].copy()
        for j in range(cross_point):
            c_index = state[parents[i+1]].index(state[parents[i]][j])
            offspring2[j], offspring2[c_index] = offspring2[c_index], offspring2[j]

        state[parents[i]], state[parents[i+1]] = offspring1, offspring2


In [8]:
def mutation(state, parents):
    
    '''
    randomly swaping any two cities in the chromosome
    Performs mutation on the childs created after crossover
    Parameters:
        state    : Population
        parents  : Two parents selected by roulette-wheel-selection process
    '''
    
    no_nodes = len(state[0])
    for p in parents:
        index1, index2 = np.random.randint(0, no_nodes-1), np.random.randint(0, no_nodes-1)
        while index2 == index1:
            index2 = np.random.randint(0, no_nodes-1)

        state[p][index1], state[p][index2] = state[p][index2], state[p][index1]


In [9]:
def find_optimal_tsp_path(graph):
    
    '''
    Tries to finds the optimal tour (min-cost tour) given the condition that each node has to visited only once
    Parameters:
        graph : The actual graph
    Prints optimal path accounted so far
    '''
#     we can run for max geneartions also, here we used true loop till keyboard interupt
    max_generations = 100000
    actual_generations = 0
    pop_size = int(input("Enter the intial population size : "))
    print()
#     max_parent = int(input("Enter the no. of parents selected to create child for next generation : "))
#     print("\n")
    
#     states of genetic algorithm i.e chromosomes
    ga_state = list()
    initialize_population(list(graph.keys()), ga_state, pop_size)
    optimal_so_far, optimal_path = 0, None

    print("Execution starts -----------------------------------------------")
    try:
        while True:
            actual_generations  = actual_generations  + 1

            fitness = []
            for i in range(pop_size):
                fitness.append(fitness_value(graph, ga_state[i]))
#             for i in range(pop_size):
#                 print(fitness[i])

            max_fitness = max(fitness)
            max_fitness_index = fitness.index(max(fitness))

            if max_fitness > optimal_so_far:
                optimal_so_far, optimal_path = max_fitness, ga_state[max_fitness_index]
                print("Current path", ga_state[max_fitness_index], "Cost", round(1/max_fitness), "Generation Number: ",actual_generations  )
            sel_parents = roulette_wheel_selection(fitness, 2)

            crossover(ga_state, sel_parents, pop_size)

            mutation(ga_state, sel_parents)
            
# Age based-selection for next generations
        return optimal_path, round(1/optimal_so_far), actual_generations 

    except KeyboardInterrupt:
        return optimal_path, round(1/optimal_so_far), actual_generations 


In [ ]:
if __name__ == '__main__':
    '''
    Main Function
    '''
    graph = defaultdict(dict)
    graph_input = None
#     graph_input = [('A', 'B', 20), ('B', 'D', 34), ('C', 'D', 12), ('A', 'C', 42), ('A', 'D', 35), ('B', 'C', 30)]
#     graph_input = [('A', 'B', 10), ('B', 'D', 25), ('C', 'D', 30), ('A', 'C', 15), ('A', 'D', 20), ('B', 'C', 35)]

# four methods for graph input

#     Creating the graph
    if graph_input is not None:
        for n1, n2, c in graph_input:
            adding_new_edge_to_graph(graph, n1, n2, c)
#     else:
#         print("\nEnter source node, destination node, cost (Space seperated. One record in each line. And put a '/' as EOF) : ")
#         line = input()
#         while line != '/':
#             elem = line.split()
#             adding_new_edge_to_graph(graph, elem[0], elem[1], int(elem[2]))
#             line = input()
            
    else:
        city=int(input("enter number of cities : "))
        max_dist=int(input("enter max distance allowed : "))
        for i in range(1,city+1):
            for j in range(i+1, city+1):
                dist=random.randint(1, max_dist)
                adding_new_edge_to_graph(graph,i,j,dist)
            '''
            Taking input of graph from file directly
            '''
#     else:
#         dirname = os.getcwd()
#         filename = os.path.join(dirname,'tsp_input.txt')
#         with open(filename,'r') as f:
#             line = f.readline()
#             while line != '/':
#                 elem = line.split()
#                 adding_new_edge_to_graph(graph, elem[0], elem[1], int(elem[2]))
#                 line = f.readline()

    if len(graph) != 0:
        # Solving the TSP Problem via genetic algorithm
        solution = find_optimal_tsp_path(graph)
        print("\n------------------------------\nAfter", solution[2], "generations : -")
        print("Optimal path till now -")
        print(solution[0])
        print("Optimal Cost till now -")
        print(solution[1])
        print("------------------------------")
    else:
        print("Graph is empty")

enter number of cities : 50
enter max distance allowed : 20
Enter the intial population size : 100

Execution starts -----------------------------------------------
Current path [5, 11, 45, 31, 3, 25, 19, 32, 34, 30, 10, 35, 8, 47, 26, 23, 14, 27, 40, 37, 18, 41, 16, 38, 21, 13, 6, 2, 24, 9, 29, 12, 20, 39, 44, 43, 33, 48, 17, 50, 46, 36, 28, 15, 49, 7, 22, 42, 1, 4] Cost 450 Generation Number:  1
Current path [11, 4, 15, 18, 2, 45, 19, 28, 44, 27, 40, 43, 42, 16, 17, 10, 8, 34, 26, 23, 39, 48, 29, 35, 46, 30, 31, 38, 20, 33, 47, 3, 25, 13, 50, 14, 1, 7, 22, 37, 21, 32, 49, 9, 36, 24, 41, 5, 12, 6] Cost 448 Generation Number:  20
Current path [41, 4, 15, 21, 2, 26, 19, 28, 44, 27, 40, 43, 45, 16, 11, 10, 8, 37, 38, 22, 20, 24, 5, 48, 29, 49, 36, 46, 30, 47, 17, 6, 39, 14, 31, 50, 25, 32, 33, 13, 34, 23, 35, 18, 3, 1, 42, 9, 12, 7] Cost 432 Generation Number:  24
Current path [32, 26, 16, 23, 21, 3, 18, 13, 14, 29, 27, 43, 20, 34, 6, 11, 19, 24, 12, 47, 36, 40, 48, 10, 49, 2, 9, 15, 37,